# Exploring propagation of record changes across databases and annotations

In [ ]:
# For each record in SwissProt
#    Extract Taxa
#    Extract ProteinName and Subname
#    Extract 

In [1]:
# Imports
from Bio import SeqIO
import gzip
import progressbar

In [2]:
# Index UniProt proteins so that we can access via their accessions. 
#  - Only needs to be run once. Takes a few minutes. 

data_dir = '../data/'
up_gz = data_dir+'uniprot_sprot.dat.bgz'
up_db = data_dir+'uniprot_sprot.dat.biopy_db'
uniprot = SeqIO.index_db(index_filename=up_db, filenames=up_gz, format="swiss")


In [13]:
# Index gb_files
gb_file=data_dir+"gbbct60.seq.gz"
#h = gzip.open(gb_file, "rt")
#r =SeqIO.parse(h, "gb").__next__()

In [28]:
# Create a mapping from GenBank and RefSeq to SwissProt record. 
from datetime import datetime
from netdbqual import utils; classify_acc(acc)
print(datetime.now().strftime("%H:%M:%S"))
gb_sp_map={id.split(":")[1]:r  for r in uniprot for id in uniprot[r].dbxrefs if id[0:4]=="EMBL"}
print(datetime.now().strftime("%H:%M:%S"))
rs_sp_map={id.split(":")[1]:r  for r in uniprot for id in uniprot[r].dbxrefs if id[0:6]=="RefSeq"}

def get_dbxrefs(r):
    return [id for id in r.dbxrefs if id[0:6]=="RefSeq" or id[0:4]=="EMBL"]

print(datetime.now().strftime("%H:%M:%S"))
dbxref_tuples=[{db_acc.split(":")[0]:(db_acc.split(":")[1], uniprot[acc])}  for acc in uniprot for db_acc in get_dbxrefs(uniprot[acc]) ]

print(datetime.now().strftime("%H:%M:%S"))
gb_sp_map={accs['EMBL'] for accs in dbxref_tuples if 'EMBL' in accs}
rs_sp_map={accs['RefSeq'] for accs in dbxref_tuples if 'RefSeq' in accs}
print(datetime.now().strftime("%H:%M:%S"))


12:56:07
15:29:43
16:00:24


KeyboardInterrupt: 

In [4]:
import re 
exec(open("../netdbqual/utils.py").read())

gb_file="gbbct15.seq.gz"
gbs_iter=SeqIO.parse(gzip.open(data_dir + gb_file, "rt"), "gb")
#gbs_cut=[gbs_iter.__next__() for i in range(300)]

#For each genome sequence,
gb_file_no_ext=gb_file.split(".")[0]


In [27]:
exec(open("../netdbqual/utils.py").read())
import glob
from os import path
gb_file="gbbct*.seq.gz"


def parse_gb_protein_gff(p):

    if 'molecule_type' not in p.annotations:
        return ([],[])


    prod=p.description.split(' [')[0] 

    if 'organism' not in p.annotations:
        return  ([],[])

    db=classify_acc(p.id)[0]
    if db!='genbank':
        a=1
        print("XXX")

    id,seq_ver=p.id.split('.')

    node=",".join([       
                p.id,  
                seq_ver,           
                prod,
                p.annotations['organism'],
                db,
                str(p.seq)
            ])
    return ([node], [])


def parse_gb_protein(p):

    if p.type!="CDS" or 'pseudo'  in p.qualifiers or 'pseudogene'  in p.qualifiers:
        return ([],[])

    if 'protein_id' not in p.qualifiers:
        return  ([],[])

    prod=""
    if 'product' in p.qualifiers:
        prod=p.qualifiers['product'][0] 

    if 'organism' not in r.annotations:
        return  ([],[])

    db=classify_acc(p.qualifiers['protein_id'][0])[0]
    if db!='genbank':
        a=1
        print("XXX")

    id,seq_ver=p.qualifiers['protein_id'][0].split('.')
    node=",".join([       
                p.id,  
                seq_ver,            
                prod,
                r.annotations['organism'],
                classify_acc(p.qualifiers['protein_id'][0])[0],
                p.qualifiers['translation'][0]     
            ])
    return (node, [])


def parse_gb_nuc(r):
    # look at all CDS 
    print(j)
    nodes=[]
    edges=[]

    for p in r.features:
        node,edge=parse_gb_protein(p)
        nodes=nodes+node
        edges=edges+edges
    return(nodes, edges)

#for i,f in enumerate(glob.glob(data_dir + gb_file)):
#f="../../220411_ExamineFunction/data/EC3_4_11_4/EC_3_4_11_4_SP_sequence.gpk"
f=data_dir + "tmp_EC_3_4_11_4_partial_sequence.gpk"
if 1:
    #gbs_iter=SeqIO.parse(gzip.open(f, "rt"), "gb")
    gbs_iter=SeqIO.parse(f, "gb")
    gb_file_no_ext=path.basename(f).split(".")[0]
    print(i)
    
    nodes=[]
    edges=[]
    with open(data_dir + gb_file_no_ext+'_node.csv', "w") as node_file:
        with open(data_dir + gb_file_no_ext+'_edge.csv', "w") as edge_file:            
            for j,r in enumerate(gbs_iter):
                # look at all CDS 
                print(j)
                (db,seq_type)=classify_acc(r.id)
                if seq_type=="nuc":
                    nodes,edges=parse_gb_nuc(r)
                elif seq_type=="protein":                    
                    nodes,edges=parse_gb_protein_gff(r)
                else:
                    print("Problem")
                    sdfsdfsd



                node_file.writelines(nodes)
                

81
0
1
2
3
4
5
6
7
8
9


In [ ]:
# For each set of genbank records (gb*.seq.gz)
#  Extract record
#  Download history
#  